In [6]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk, os, re, string, collections
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from  imageio import imread
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
import multidict as multidict #pip install multidict
import itertools
import csvtomd
import re
from collections import Counter
#%matplotlib inline
import spacy
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# 1. Limpieza y conversión de datos

In [2]:
dtype_test = {"id": np.int32, "keyword": "category"}
test = pd.read_csv("original_data/test.csv", dtype = dtype_test)

In [3]:
dtype_train = {"id": np.int32, "keyword": "category", "target" : bool}
train = pd.read_csv("original_data/train.csv", dtype = dtype_train, encoding='UTF_8')

In [4]:
target_train = train.target

## 1.1 Keywords

In [5]:
nlp = spacy.load('en', disable=['parser', 'ner'])

def lemmatize_text(text):
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer  = nltk.stem.WordNetLemmatizer()
    if text == '' or pd.isnull(text):
        return text
    else:
        wordList = [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
        s = ' '
        return s.join(wordList)
    
def lemmatize_text_v2(text):
    if text == '' or pd.isnull(text):
        return text
    else:
        doc = nlp(text)
        return " ".join([token.lemma_ for token in doc])

def clean_keyword_field(df):
    df["keyword"] = df["keyword"].str.replace('%20',' ')
    df_sin_keywords_nulos = df.dropna(subset = ['keyword'])
    keywords = np.unique(df_sin_keywords_nulos.keyword)
    for keyword in keywords:
        df.loc[(df['text'].str.contains(keyword, case=False) & df['keyword'].isnull()), 'keyword'] = keyword
    #df['keyword'] = df['keyword'].apply(lemmatize_text_v2)
    return df

NameError: name 'spacy' is not defined

## 1.2 Text

### Strict (for tags)

In [ ]:
def getStopwordsList():
    fileNamesList = ["texts/99webtools.txt", "texts/atire_ncbi.txt", "texts/atire_puurula.txt", "texts/azure.txt", 
                 "texts/bbalet.txt", 
                 "texts/bow_short.txt", "texts/choi_2000naacl.txt", "texts/cook1988_function_words.txt", 
                 "texts/corenlp_acronym.txt", 
                 "texts/corenlp_hardcoded.txt", "texts/corenlp_stopwords.txt", "texts/datasciencedojo.txt", 
                 "texts/deeplearning4j.txt", 
                 "texts/dkpro.txt", "texts/mongodb.txt", "texts/galago_inquery.txt", "texts/gate_keyphrase.txt", 
                 "texts/gensim.txt", 
                 "texts/glasgow_stop_words.txt", "texts/indri.txt", "texts/kevinbouge.txt", "texts/lexisnexis.txt",
                 "texts/lingpipe.txt", 
                 "texts/mallet.txt", "texts/mysql_innodb.txt", "texts/mysql_myisam.txt", "texts/galago_rmstop.txt", 
                 "texts/atire_ncbi.txt", 
                 "texts/galago_rmstop.txt", "texts/nltk.txt", "texts/okapiframework.txt", "texts/okapi_cacm_expanded.txt", 
                 "texts/onix.txt", 
                 "texts/ovid.txt", "texts/postgresql.txt", "texts/pubmed.txt", "texts/quanteda.txt", "texts/r_tm.txt", 
                 "texts/ranksnl_large.txt", 
                 "texts/reuters_wos.txt", "texts/rouge_155.txt", "texts/scikitlearn.txt", "texts/smart.txt", 
                 "texts/snowball_expanded.txt", 
                 "texts/spacy.txt", "texts/spark_mllib.txt", "texts/sphinx_mirasvit.txt", "texts/t101_minimal.txt", 
                 "texts/taporware.txt", 
                 "texts/terrier.txt", "texts/tonybsk_1.txt", "texts/tonybsk_6.txt", "texts/voyant_taporware.txt", 
                 "texts/weka.txt", 
                 "texts/xapian.txt", "texts/xpo6.txt", "texts/zettair.txt"]
    stopwordsList = []
    for fileName in fileNamesList:
        file = open(fileName, "r")
        for line in file:
            stripped_line = line. strip()
            line_list = stripped_line
            if line_list not in stopwordsList:
                stopwordsList.append(line_list)
        file.close()
    return stopwordsList

stopwordsList = getStopwordsList()

print(len(stopwordsList))

In [ ]:
def clean_text_strict(text):
    text = text.lower()
    text = text.replace('\d+', '')
    invalid_chars = ['#','|','@','!','?','-','_','[',']','%','&',':','.',',',"''",'/','https','(','//t',')','http',
                 ';','\'']
    for char in invalid_chars:
        if char in text:
            text = text.replace(char,' ')
    #removes url and tags
    text = re.sub(r"(?:\@|https?\://)\S+", " ",  text)
    return text

### Non strict (for special characters, hashtags, etc)

In [ ]:
def clean_text_non_strict(text): 
    tw = " "
    words = text.split(' ')
    for word in words:
        word = ''.join(filter(lambda x: x in set(string.printable), word))
        word = word.replace("\n"," ")
        word = word.replace('û',"")
        word = word.replace('Û',"")
        #word = word.replace('_','')
        #word = word.replace("\"",'')
        #word = word.strip('.')
        #word = word.strip(',')
        #word = word.strip(':')
        tw += word + " "
    return tw

## 1.3 Location

In [ ]:
#FUNCIONES AUXILIARES: Limpieza de tweet

# Función que saca los links

def quitar_link_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if (("http" not in w) and ("https" not in w)):
            res.append(w)
    return (" ").join(res)

# Función que saca las menciones

def quitar_mencion_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if("@" not in w):
            res.append(w)
    return (" ").join(res)

def quitar_simbolo(tweet, simbolo):
    text = ""
    for c in tweet:
        if(c!=simbolo):
            text+=c
    return text

def agregar_espacio_ente_comas(tweet):
    
    tweer_sin_coma = (" ").join(tweet.split(","))

    return (" ").join(tweer_sin_coma.split("."))

def capitalize_each_word(string):
    
    text = ""
    for w in string.split(" "):
        if(len(w)>3):
            text += w.capitalize()+" "
    return text

def limpiar_tweet_location(tweet):
    res = tweet
    func = [quitar_mencion_twitter, quitar_link_twitter, agregar_espacio_ente_comas, lambda x: quitar_simbolo(x, "#"), capitalize_each_word]
    for f in func:
        res = f(res)
    return res

def to_string(a_list, delimeter):
    res = ""
    
    for e in sorted(filter(None, a_list)):
        res += str(e)+delimeter
    
    if(len(res)==0): return None
    return res[: -len(delimeter)]

In [ ]:
# FUNCIONES AUXILIARES: Analisis geografico

from geotext import GeoText
import pycountry

def get_country_pycountry(text):
    for country in pycountry.countries:
        # Handle both the cases(Uppercase/Lowercase)
        if str(country.name).lower() in str(text).lower():
            return GeoText(country.name).country_mentions

def get_country_geotext(text):
    return GeoText(text).country_mentions

def get_country(text):
    set1 = set(get_country_geotext(text))
    countries = get_country_pycountry(text)
    if(not countries):
        return list(set1)
    return list(set1.union(set(countries)))

### Location in text

In [ ]:
# Creo dataframe y limpio la location

location_in_text = train[["id", "text"]].copy()
location_in_text['text'] = location_in_text['text'].apply(lambda x: limpiar_tweet_location(x))

In [ ]:
#Creo columnas con informacion

location_in_text["cities"] = location_in_text["text"].apply(lambda x: GeoText(str(x)).cities)
location_in_text["cities_str"] = location_in_text["cities"].apply(lambda x: to_string(x,","))

location_in_text["country"] = location_in_text["text"].apply(lambda x: get_country(x))
location_in_text["country_str"] = location_in_text["country"].apply(lambda x: to_string(x,","))

location_in_text["has_country"] = location_in_text["country"].apply(lambda x: len(x)!=0)

In [ ]:
# Selecciono las columnas más útiles para crear features

location_in_text = location_in_text[["id","cities_str","country_str","has_country"]]
location_in_text.columns = ["id", "has_location", "cities", "country"]

In [ ]:
location_in_text.head()

### Location label info

In [ ]:
# Creo dataframe y limpio la location

location = train[["id", "location","target"]].copy().fillna("")
location["location"] = location["location"].apply(lambda x: limpiar_tweet_location(x))

In [ ]:
#Creo columnas con informacion

location["has_location"] = location["location"]!=""

location["cities"] = location["location"].apply(lambda x: GeoText(str(x)).cities)
location["cities_str"] = location["cities"].apply(lambda x: to_string(x,","))

location["country"] = location["location"].apply(lambda x: get_country(x))
location["country_str"] = location["country"].apply(lambda x: to_string(x,","))

In [ ]:
location_aux = location[["country_str", "cities_str"]].isnull()[520:525]
location["city_country_not_found"] = location_aux["country_str"] & location_aux["cities_str"]

In [ ]:
# Selecciono las columnas más útiles para crear features

location_info = location[["id", "has_location", "cities_str", "country_str", "city_country_not_found"]]
location_info.columns = ["id", "has_location", "cities", "country", "city_country_not_found"]

In [ ]:
location_info.head()

## 1.4 Links

In [ ]:
links = pd.read_csv("new_data/links.csv", encoding='UTF_8').set_index("id")

In [ ]:
import ast

links['http']= links['http'].apply(lambda x: ast.literal_eval(x))

In [ ]:
#Funciones auxiliares

def get_dominio(links):
    res= []
    for l in links:
        if(l==None):
            res.append(None)
        else:
            dom = l.split("/")
            res.append(dom[2])
    return res

def type_http(links):
    res = []
    for l in links:
        if(l==None):
            res.append(None)
        else:
            dom = l.split(":")
            res.append(dom[0])
    return res

def count_None(a_list):
    count = 0
    for l in a_list:
        if(l==None):
            count+=1
    return count

def link_encription_generalize(link_encript):
    
    has_http = False
    has_https = False
    
    for type_http in link_encript:
        if type_http==None: 
            continue
        elif type_http== "https":
            has_https = True
        else:
            has_http = True
    
    if(has_http and has_https):
        return "both"
    elif(has_http):
        return "http"
    elif(has_https):
        return "https"
    else:
        return None
    
def to_string(a_list, delimeter):
    res = ""
    
    for e in sorted(filter(None, a_list)):
        res += str(e)+delimeter
    
    if(len(res)==0): return None
    return res[: -len(delimeter)]

In [ ]:
links["links_domain"] = links["http"].apply(lambda x: get_dominio(x))
links["links_encription"] = links["http"].apply(lambda x: type_http(x))
links["links_encription_type"] = links["links_encription"].apply(lambda x: link_encription_generalize(x))
links["cant_failed_links"] = links["http"].apply(lambda x: count_None(x))
links["links_domain_str"] = links["links_domain"].apply(lambda x: to_string(x,","))

## 1.5 Tags

### Limpieza de los campos para obtener tags

In [ ]:
# FUNCIONES AUXILIARES

def prettify(word_list):
    wl = []
    for word in word_list:
        word = re.sub(r'[^A-Za-z]', " ",  word)
        word = re.sub(r'\b\w\b',' ', word)
        word = re.sub(r'\s+', ' ', word)
        wl.append(word.strip())
    return wl

In [ ]:
# Creo el dataframe
df_tags = train[['id','text','target']].copy(deep=True)

# Limpio el dataframe
df_tags['new'] = df_tags["text"].apply(lambda x: clean_text_strict(x))

In [ ]:
#  Eliminacion de terminos, caracteres y stopwords innecesarios para generar los tags
#convierte de list a string

df_tags['new'] = df_tags['new'].apply(lambda x: [item for item in x.split() if item not in stopwordsList])
df_tags.new = df_tags.new.apply(prettify)

df_tags.head()

### Tokenize

In [ ]:
# FUNCION AUXILIAR
def lemmatize_text2(word_list):
    text = ' '
    text = text.join(word_list)
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

df_tags['new_tokenized'] = df_tags.new.apply(lemmatize_text2)
df_tags.new_tokenized = df_tags.new_tokenized.apply(prettify)

df_tags.head()

### Analisis de ocurrencias

In [ ]:
def getText(word_list):
    text = ' '
    text = text.join(word_list)
    return text

#metodo para analizar ocurrencias y cantidades
a = df_tags['new_tokenized'].apply(getText).str.lower().str.cat(sep=' ')

words = nltk.tokenize.word_tokenize(a)
word_dist = nltk.FreqDist(words)

this_col = pd.Series(['word','frequency'])

rslt = pd.DataFrame(word_dist.most_common(400),columns = this_col)

In [ ]:
frequent_words = rslt ['word']
unique_frequent_words = set(frequent_words)

### Se genera columna con tags a partir de una cantidad de ocurrencias destacadas

In [ ]:
def word_finder(x):
    x = set(x)
    return list(unique_frequent_words.intersection(x))

In [ ]:
df_tags['tags'] = df_tags.new_tokenized.apply(word_finder)
tags_info = df_tags[['id','target','text','tags']]

tags_info.head()

# 2. Features

## 2.1 Keywords

In [ ]:
df_keywords = clean_keyword_field(train)

In [ ]:
df_keywords_lemmatized = df_keywords.copy()
df_keywords_lemmatized.keyword = df_keywords.keyword.apply(lambda x: lemmatize_text_v2(x))

In [ ]:
def toText(listOfWords):
    s=' '
    return s.join(listOfWords)

def textContainsKeyword(row):
    if row.keyword == '' or pd.isnull(row.keyword):
        return False
    else:
        return row.keyword.lower() in row.text.lower()
    
def getKeywords(df):
    df_sin_keywords_nulos = df.dropna(subset = ['keyword'])
    return np.unique(df_sin_keywords_nulos.keyword)

def textContainsKeywordAsHashtag(row):
    hashtags = []
    tweet = row.text.lower()
    if row.keyword == '' or pd.isnull(row.keyword):
        return False
    else:
        for word in tweet.split(' '):
            if (len(word)>1) and (word[0] == '#'):
                hashtags.append(word[1:])
        return row.keyword.lower() in hashtags

def linkContainsKeyword(row):
    links   = row.http
    keyword = row.keyword
    if row.keyword == '' or pd.isnull(row.keyword):
        return False
    if (type(links)==list):
        for link in links:
            if link=='' or pd.isnull(link):
                continue
            if keyword in link:
                return True
    else:
        return False
    return False

In [ ]:
all_keywords = pd.get_dummies(df_keywords_lemmatized[['id','keyword']],'keyword')
all_keywords.head()

In [ ]:
#df_keywords = df_train.copy()

In [ ]:
#df_sin_keywords_nulos = df_keywords.dropna(subset=['keyword'])
#%time df_keywords_not_in_text = df_sin_keywords_nulos[~df_sin_keywords_nulos['text'].str.contains("|".join(df_sin_keywords_nulos['keyword']),case=False)]
#%time df_sin_keywords_nulos[df_sin_keywords_nulos.apply(lambda x: x.keyword in x.text, axis=1)]

In [ ]:
df_keywords['text_contains_keyword'] = df_keywords.apply(textContainsKeyword,axis=1)
df_keywords.head()

In [ ]:
df_keywords['has_keyword'] = ~df_keywords.keyword.isnull()

In [ ]:
keywords = getKeywords(df_keywords)
keywords

In [ ]:
keywords_lemmatized = getKeywords(df_keywords_lemmatized)
keywords_lemmatized

In [ ]:
def countKeywords(tweet):
    quantity = 0
    tweet = re.sub('[^a-zA-Z\ ]+', "", tweet).lower()
    for word in tweet.split(' '):
        #word = re.sub('[^a-zA-Z]+', '', word)
        if word in keywords:
            quantity+=1
    return quantity

def getHiddenKeywords(tweet):
    k = []
    tweet = re.sub('[^a-zA-Z\ ]+', "", tweet).lower()
    for keyword in keywords:
        if keyword in tweet:
            k.append(lemmatize_text_v2(keyword))
    return k

df_keywords['keywords_quantity'] = df_keywords.text.apply(countKeywords)
df_keywords['keywords_in_text'] = df_keywords.text.apply(getHiddenKeywords)
df_keywords.head()

In [ ]:
df_keywords['keywords_mean'] = df_keywords.text.apply(countKeywords)/keywords.size
df_keywords.head()

In [ ]:
df_keywords['keyword_is_hashtag'] = df_keywords.apply(textContainsKeywordAsHashtag,axis=1)
df_keywords.head()

In [ ]:
# busco frecuencia de cada keyword para asignar un 'peso'
df_keywords['keyword_frequency'] = df_keywords_lemmatized.groupby('keyword')['id'].transform('count') / keywords_lemmatized.size
df_keywords.head()

In [ ]:
just_links = links.reset_index()[['http','id']]
df_keywords_links = df_keywords.merge(just_links, on='id', how='left')
df_keywords_links.head()

In [ ]:
df_keywords['link_contains_keyword'] = df_keywords_links.apply(linkContainsKeyword,axis=1)

In [ ]:
df_keywords['keyword_lemmatized'] = df_keywords_lemmatized['keyword']

In [ ]:
mlb = MultiLabelBinarizer()

df_keywords = df_keywords.join(pd.DataFrame(mlb.fit_transform(df_keywords.pop('keywords_in_text')),
                          columns=mlb.classes_,
                          index=df_keywords.index).add_prefix('K_'))
df_keywords.head()

In [ ]:
df_keywords.drop(columns=['location', 'text'])

In [ ]:
#corrmap = df_keywords.corr()
#fig = plt.figure(figsize=(20,10))

#sns.heatmap(corrmap, vmax=.8, square=True)
#plt.show()

## 2.2 Text

In [ ]:
#Funciones auxiliares
stopword = stopwords.words('english')

def stop(text):
    return [w for w in text.split() if w in stopword]

def length(text):
    return(np.mean([len(w) for w in text.split()]))

def punct(text):
    return [c for c in text if c in string.punctuation]

def title(text):
    return [w for w in text.split() if w.istitle()]

def upper_list(text):
    return [w for w in text.split() if w.isupper()]

def lower_list(text):
    return [w for w in text.split() if w.islower()]

def count_special_char(text):
    special_char = 0
    text = text.replace(" ", "")
    for i in range(0, len(text)):
        ch = text[i]
        if (ch.isalpha()):  
            continue
        elif (ch.isdigit()):
            continue
        else:
            special_char += 1
    return special_char

def syllables(text):
    count = 0
    vowels = 'aeiouy'
    for word in text:
        word= word.lower()
        if word[0] in vowels:
            count +=1
        for index in range(1,len(word)):
            if word[index] in vowels and word[index-1] not in vowels:
                count +=1
        if word.endswith('e'):
            count -= 1
        if word.endswith('le'):
            count+=1
        if count == 0:
            count +=1
    return count

def quitar_link_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if (("http" not in w) and ("https" not in w)):
            res.append(w)
    return (" ").join(res)

def quitar_mencion_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if("@" not in w):
            res.append(w)
    return (" ").join(res)

def getHashtags(tweet):
    pat = re.compile(r"#(\w+)")
    return pat.findall(tweet)

def getTaggedUsers(tweet):
    pat = re.compile(r"@(\w+)")
    return pat.findall(tweet)

def agregar_features_cantidad_palabras(df_og):
    df_og['text'] = df_og['text'].apply(lambda x: quitar_link_twitter(x))
    df_og['text'] = df_og['text'].apply(lambda x: quitar_mencion_twitter(x))
    palabras_list = [df_og]
    for df in palabras_list:
        # Número de palabras usadas
        df['#palabras'] = df['text'].apply(lambda x: len(x.split()))
        # Número de palabras únicas
        df['#palabras_unicas'] =  df['text'].apply(lambda x: len(set(x.split())))
        # Número de caracteres
        df['#caracteres'] =  df['text'].apply(lambda x: len(x))
        # Número de stopwords
        df['#stopwords'] = df['text'].apply(lambda x: len(stop(x)))
        # Número de caracteres de puntuación
        df['#puntuacion'] = df['text'].apply(lambda x: len(punct(x)))
        # Número de palabras Capitalizadas
        df['#capitalize'] = df['text'].apply(lambda x: len(title(x)))
        # Número de palabras MAYUSCULAS
        df['#mayusculas'] = df['text'].apply(lambda x: len(upper_list(x)))
        # Número de silabas
        df['#silabas'] = df['text'].apply(lambda x: syllables(x))
        # Promedio de longitud del tweet
        df['promedio_len_word'] = df['text'].apply(lambda x: length(x))
        # Número de caracteres especiales
        df['#caracteres_especiales'] = df['text'].apply(lambda x: count_special_char(x))
    return df

def getUniqueHashtagsList(hashtags):
    hashtagList = []
    for hashtag in hashtags:
        for h in hashtag:
            if h not in hashtagList:
                hashtagList.append(h.lower())
    return hashtagList

def getHashtagsList(hashtags):
    hashtagList = []
    for hashtag in hashtags:
        for h in hashtag:
            hashtagList.append(h.lower())
    return hashtagList

def getMostFrequentHashtag(hashtags):
    maxValue = 0
    finalHashtag = ''
    if (len(hashtags) > 0):
        for hashtag in hashtags:
            hashtag = hashtag.lower()
            if (hashtagDict.get(hashtag) > maxValue):
                finalHashtag = hashtag
                maxValue = hashtagDict.get(hashtag)

    return finalHashtag

def getHashtagFrequency(hashtag):
    if hashtag != '':
        return hashtagDict.get(hashtag)/len(hashtagDict)
    else:
        return 0
    
def getUniqueUsersList(users):
    userList = []
    for user in users:
        for u in user:
            if u not in userList:
                userList.append(u.lower())
    return userList

def getUsersList(users):
    userList = []
    for user in users:
        for u in user:
            userList.append(u.lower())
    return userList

def hasRelevantTaggedUser(users):
    minValue = 5
    #finalUser = ''
    if (len(users) > 0):
        for user in users:
            user = user.lower()
            if (userDict.get(user) > minValue):
                return True
    return False

In [ ]:
df_text = train.copy()

In [ ]:
df_text.text = df_text.text.apply(clean_text_non_strict)
df_text['tagged_users'] = df_text.text.apply(lambda x: getTaggedUsers(x))
df_text['#tagged_users'] = df_text.tagged_users.apply(lambda x: len(x))
df_text['has_tagged_user'] = df_text['#tagged_users']>0
df_text.head()

In [ ]:
users = getUniqueUsersList(df_text.tagged_users)
userDict = Counter(getUsersList(df_text.tagged_users))
df_text['has_relevant_tagged_user'] = df_text.tagged_users.apply(lambda x:hasRelevantTaggedUser(x))
df_text.head()

In [ ]:
df_text = agregar_features_cantidad_palabras(df_text)
df_text.head()

In [ ]:
# Binning numerical data (esta super arbitrario esto)
df_text['#palabras_binned'] = pd.qcut(df_text['#palabras'], q=4, labels=False)
df_text['#palabras_unicas_binned'] = pd.qcut(df_text['#palabras_unicas'], q=4, labels=False)
df_text['#caracteres_binned'] = pd.qcut(df_text['#caracteres'], q=5, labels=False)
df_text['#stopwords_binned']  = pd.qcut(df_text['#stopwords'], q=3, labels=False)
df_text['#puntuacion_binned'] = pd.qcut(df_text['#puntuacion'], q=3, labels=False)
df_text['#capitalize_binned'] = pd.qcut(df_text['#capitalize'], q=3, labels=False)
df_text['#mayusculas_binned'] = pd.qcut(df_text['#mayusculas'], q=2, labels=False, duplicates='drop')
df_text['#silabas_binned']    = pd.qcut(df_text['#silabas'], q=4, labels=False)
df_text['#caracteres_especiales_binned']    = pd.qcut(df_text['#caracteres_especiales'], q=3, labels=False)
df_text.head()

In [ ]:
df_text['hashtags'] = df_text.text.apply(lambda x: getHashtags(x))
df_text['#hashtags'] = df_text.hashtags.apply(lambda x: len(x))
df_text['has_hashtag'] = df_text['#hashtags']>0
hashtags = getUniqueHashtagsList(df_text.hashtags)
hashtagDict = Counter(getHashtagsList(df_text.hashtags))

df_text.head()

In [ ]:
df_text['relevant_hashtag'] = df_text.hashtags.apply(lambda x: getMostFrequentHashtag(x))
df_text.head()

In [ ]:
df_text['freq_relevant_hashtag'] = df_text.relevant_hashtag.apply(lambda x:getHashtagFrequency(x))
df_text.head()

## 2.3 Location

### Same location in label and in text

In [ ]:
#Merge info location in text and in columnn "location"
location_merged = location_in_text.merge(location_info, how="left", on="id",suffixes = ('_in_label', '_in_text'))

In [ ]:
location_merged.head()

### Location label countries

In [ ]:
# Para aquellos tweets con más de dos paises, separo en columnas

countries = location['country_str'].str.split(",", expand = True)

countries["id"] = location["id"]
countries.set_index("id")

countries1 = pd.get_dummies(countries[0],"Country")
countries2 = pd.get_dummies(countries[1],"Country")

# Uno las columnas
for country in countries1.columns:
    if(country in countries2.columns):
        countries1[country] = countries1[country] + countries2[country]
for country in countries2.columns:
    if(country in countries1.columns):
        countries1[country] = countries2[country]

In [ ]:
countries1.head(2)

### BOW: Coutry

In [ ]:
bow_country_index = location['country_str'].dropna().index
corpus = list(location['country_str'].dropna())

In [ ]:
country_vectorizer = CountVectorizer()

bow_country = country_vectorizer.fit_transform(corpus).todense()
df_bow_country = pd.DataFrame(bow_country)
df_bow_country.set_index(bow_country_index)
df_bow_country.columns = country_vectorizer.get_feature_names()

In [ ]:
df_bow_country.head()

### Tf-Idf: Country

In [ ]:
# Without Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_country=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_country.set_index(bow_country_index)

df_tf_idf_country.head()

In [ ]:
# With Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_smooth_country=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_smooth_country.set_index(bow_country_index)

df_tf_idf_smooth_country.head()

## 2.4 Links

#### Features con cantidades

In [ ]:
links_info = links[["links_cant", "cant_failed_links", "links_encription_type"]].copy()

In [ ]:
links_info.head()

#### Features con cantidad de apariciones de n dominios más usuales

In [ ]:
# n cantidad de links
n_most_common_links = 20

In [ ]:
# Dejo en cada columna un links

links_column = links['links_domain_str'].str.split(",", expand = True)
links_column[105:110]

In [ ]:
links_cant_domain = pd.get_dummies(links_column[0],"domain")

In [ ]:
# Sumo las columnas de acuerdo al link

unique_domains = set(links_cant_domain.columns)

for i in range(1, len(links_column.columns)):
    n_links_pd = pd.get_dummies(links_column[i],"domain")
    for domain in unique_domains:
        if(domain in set(n_links_pd.columns)):
            links_cant_domain[domain] = links_cant_domain[domain] + n_links_pd[domain]

In [ ]:
links_cant_domain.head()

In [ ]:
# n most common domains
n_domains = 20

In [ ]:
# calculo los dominios más usuales
unique_domains_ordered = links_column[0].value_counts().index
columns_n_unique_links = ["domain_"+link for link in unique_domains_ordered[:n_domains]]

most_common_domain_cant = links_cant_domain[columns_n_unique_links]

In [ ]:
most_common_domain_cant.head()

### BOW: Dominios

In [ ]:
bow_domain_index = links['links_domain_str'].dropna().index
corpus = list(links['links_domain_str'].dropna())


In [ ]:
domain_vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)

bow_domain = domain_vectorizer.fit_transform(corpus).todense()
df_bow_domain = pd.DataFrame(bow_domain)
df_bow_domain.set_index(bow_domain_index)
df_bow_domain.columns = domain_vectorizer.get_feature_names()

In [ ]:
df_bow_domain.head()

### Tf-Idf:  Domain

In [ ]:
bow_domain_index = links['links_domain_str'].dropna().index
corpus = list(links['links_domain_str'].dropna())

In [ ]:
# Without Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_domain=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_domain.set_index(bow_domain_index)

df_tf_idf_domain.head()

In [ ]:
# With Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_smooth_domain=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_smooth_domain.set_index(bow_domain_index)

df_tf_idf_smooth_domain.head()

## 2.5 Tags

### Informacion general tags

In [ ]:
tags_info = tags_info.copy()
tags_info["cant_tags"] = tags_info["tags"].apply(lambda x: len(x))
tags_info["tags_str"] = tags_info["tags"].apply(lambda x: to_string(x,","))

In [ ]:
tags_info.head(2)

###  Nuevo feature por tag

In [ ]:
tags_columns = tags_info['tags_str'].str.split(",", expand = True)

In [ ]:

df_has_tags = pd.get_dummies(tags_columns[0],"tag")
unique_tag = set(df_has_tags.columns)

for i in range(1, len(tags_columns.columns)):
    n_links_pd = pd.get_dummies(tags_columns[i],"tag")
    for domain in unique_tag:
        if(domain in set(tags_columns.columns)):
            df_has_tags[domain] = df_has_tags[domain] + tags_columns[domain]

In [ ]:
df_has_tags["id"] = tags_info["id"]
df_has_tags = df_has_tags.set_index("id")

In [ ]:
df_has_tags.head()

### BOW: tags

In [ ]:
bow_tags_index = tags_info['tags_str'].dropna().index
corpus = list(tags_info['tags_str'].dropna())

In [ ]:
domain_vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)

bow_tags = domain_vectorizer.fit_transform(corpus).todense()
df_bow_tags = pd.DataFrame(bow_tags)
df_bow_tags.index = bow_tags_index
df_bow_tags.columns = domain_vectorizer.get_feature_names()

In [ ]:
df_bow_tags.head()

### Tf-idf: tags

In [ ]:
bow_tags_index = tags_info['tags_str'].dropna().index
corpus = list(tags_info['tags_str'].dropna())

In [ ]:
# Without Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_tags=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_tags.set_index(bow_tags_index)

df_tf_idf_tags.head()

In [ ]:
# With Smoothing:

tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(corpus)

#create dataframe
df_tf_idf_smooth_tags =pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
df_tf_idf_smooth_tags.set_index(bow_tags_index)

df_tf_idf_smooth_tags.head()